In [16]:
code = """
// C program to implement recursive Binary Search
#include <stdio.h>
  
// A recursive binary search function. It returns
// location of x in given array arr[l..r] is present,
// otherwise -1
int binarySearch(int arr[], int l, int r, int x)
{
    if (r >= l) {
        int mid = l + (r - l) / 2;
  
        // If the element is present at the middle
        // itself
        if (arr[mid] == x)
            return mid;
  
        // If element is smaller than mid, then
        // it can only be present in left subarray
        if (arr[mid] > x)
            return binarySearch(arr, l, mid - 1, x);
  
        // Else the element can only be present
        // in right subarray
        return binarySearch(arr, mid + 1, r, x);
    }
  
    // We reach here when element is not
    // present in array
    return -1;
}
  
int main(void)
{
    int arr[] = {1,2,3,4,7,9,13,24,55,56,67,88};
    int n = sizeof(arr) / sizeof(arr[0]);
    int x = 55;
    int result = binarySearch(arr, 0, n - 1, x);
    (result == -1) ? printf("Element is not present in array")
                   : printf("Element is present at index %d",
                            result);
    return 0;
}
"""

In [17]:
text_file = open("mpiBinary1.c", "w");
text_file.write(code);
text_file.close();

In [18]:
%%time
!mpiCC mpiBinary1.c

CPU times: user 1.38 ms, sys: 6.1 ms, total: 7.49 ms
Wall time: 114 ms


In [19]:
%%time
!mpirun --allow-run-as-root -np 4 ./a.out

Element is present at index 8Element is present at index 8Element is present at index 8Element is present at index 8CPU times: user 2.07 ms, sys: 7.95 ms, total: 10 ms
Wall time: 117 ms


In [20]:
code = """
#include<mpi.h>
#include<stdio.h>
#include <chrono>
#include <bits/stdc++.h>
using namespace std;
using namespace std::chrono;
#define n 12

#define key 55

int a[] = {1,2,3,4,7,9,13,24,55,56,67,88};

int a2[20];

int binarySearch(int *array, int start, int end, int value) {
    int mid;
    
    while(start <= end) {
        mid = (start + end) / 2;
        if(array[mid] == value) 
            return mid;
        else if(array[mid] > value)
            end = mid - 1;
        else
            start = mid + 1;
    }
    return -1;
}

int main(int argc, char* argv[]) {
    int pid, np, elements_per_process, n_elements_received;
    
    MPI_Status status;
    
    MPI_Init(&argc, &argv);
    
    MPI_Comm_rank(MPI_COMM_WORLD, &pid);
    MPI_Comm_size(MPI_COMM_WORLD, &np);
    
    if(pid == 0) {
        int index, i;
        
        if(np > 1) {
            for(i=1; i<np-1; i++) {
                
                index = i * elements_per_process;
                //element count
                MPI_Send(&elements_per_process, 1, MPI_INT, i, 0, MPI_COMM_WORLD);

                MPI_Send(&a[index], elements_per_process, MPI_INT, i, 0, MPI_COMM_WORLD);
            
            }
            
            index = i* elements_per_process;
            
            int elements_left = n - index;
            
            MPI_Send(&elements_left, 1, MPI_INT, i, 0, MPI_COMM_WORLD);
            
            MPI_Send(&a[index], elements_left, MPI_INT, i, 0, MPI_COMM_WORLD);
        }
        
        int position = binarySearch(a, 0, elements_per_process-1, key);
        
        if(position != -1)
          printf("Found at: %d", position);
        
        int temp;
        
        for(i=1; i<np; i++) {
            MPI_Recv(&temp, 1, MPI_INT, MPI_ANY_SOURCE, 0, MPI_COMM_WORLD, &status);
            int sender = status.MPI_SOURCE;
            
            if(temp != -1)
                printf("Found at: %d by %d", (sender*elements_per_process)+temp, sender);
        }
    }
    
    else {
        MPI_Recv(&n_elements_received, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        
        MPI_Recv(&a2, n_elements_received, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
    
        int position = binarySearch(a2, 0, n_elements_received-1, key);
        
        MPI_Send(&position, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }
    
    MPI_Finalize();
    
    return 0;
}
"""

In [21]:
text_file = open("mpiBinary.c", "w");
text_file.write(code);
text_file.close();

In [22]:
%%time
!mpiCC mpiBinary.c

CPU times: user 13 ms, sys: 4.08 ms, total: 17.1 ms
Wall time: 1.12 s


In [23]:
%%time
!mpirun --allow-run-as-root -np 4 ./a.out

Found at: 8 by 3CPU times: user 4.66 ms, sys: 4.22 ms, total: 8.87 ms
Wall time: 311 ms


In [1]:
code = """
#include<iostream>


using namespace std;
int q[100];
int visited[7];
int local_q;

void bfs(int adj_matrix[7][7], int first, int last, int q[], int n_nodes) {
    if(first==last)
      return;
    
    int cur_node = q[first++];
    cout<<"  "<<cur_node;
 
    for(int i=0; i<n_nodes; i++) {
        if(adj_matrix[cur_node][i] == 1 && visited[i] == 0){
            q[last++] = i;
            visited[i] = 1;
        }
    }
    
    bfs(adj_matrix, first, last, q, n_nodes);
}

int main() {
    int first = -1;
    int last = 0;
    int n_nodes = 7;
    
    for(int i=0; i<n_nodes; i++) {
        visited[i] = 0;
    }
    
    int adj_matrix[7][7] = {
      {0, 1, 1, 0, 0, 0, 0},
      {1, 0, 1, 1, 0, 0, 0},
      {1, 1, 0, 0, 1, 0, 0},
      {0, 1, 0, 0, 1, 0, 0},
      {0, 0, 1, 1, 0, 1, 0},
      {0, 0, 0, 0, 1, 0, 1},
      {0, 0, 0, 0, 0, 1, 0}
    };
    
    int start_node = 3;
    q[last++] = start_node;
    first++;
    visited[start_node] = 1;
 
    bfs(adj_matrix, first, last, q, n_nodes);
    
    return 0;
    
}
"""

In [2]:
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [3]:
!g++ -fopenmp code.cpp

In [4]:
%%time
!./a.out

  3  1  4  0  2  5  6CPU times: user 7.19 ms, sys: 3.34 ms, total: 10.5 ms
Wall time: 118 ms


In [5]:
code = """
#include<iostream>
#include<omp.h>

using namespace std;
int q[100];
int visited[7];
int local_q;

void bfs(int adj_matrix[7][7], int first, int last, int q[], int n_nodes) {
    if(first==last)
      return;
    
    int cur_node = q[first++];
    cout<<"  "<<cur_node;
    
    omp_set_num_threads(10);
    
    #pragma omp parallel for shared(visited)
    for(int i=0; i<n_nodes; i++) {
        if(adj_matrix[cur_node][i] == 1 && visited[i] == 0){
            q[last++] = i;
            visited[i] = 1;
        }
    }
    
    bfs(adj_matrix, first, last, q, n_nodes);
}

int main() {
    int first = -1;
    int last = 0;
    int n_nodes = 7;
    
    for(int i=0; i<n_nodes; i++) {
        visited[i] = 0;
    }
    
    int adj_matrix[7][7] = {
      {0, 1, 1, 0, 0, 0, 0},
      {1, 0, 1, 1, 0, 0, 0},
      {1, 1, 0, 0, 1, 0, 0},
      {0, 1, 0, 0, 1, 0, 0},
      {0, 0, 1, 1, 0, 1, 0},
      {0, 0, 0, 0, 1, 0, 1},
      {0, 0, 0, 0, 0, 1, 0}
    };
    
    int start_node = 3;
    q[last++] = start_node;
    first++;
    visited[start_node] = 1;
 
    bfs(adj_matrix, first, last, q, n_nodes);
    
    return 0;
    
}
"""

In [6]:
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [7]:
!g++ -fopenmp code.cpp

In [8]:
%%time
!./a.out

  3  4  1  5  2  0  6CPU times: user 5.55 ms, sys: 2.89 ms, total: 8.44 ms
Wall time: 122 ms
